In [1]:
import os
import pandas as pd
from pdf2image import convert_from_path
import numpy as np
import pytesseract
import spacy
from pytesseract import Output
import cv2
import json
from spacy.matcher import PhraseMatcher
from spacy.util import filter_spans
from spacy.tokens import DocBin
from pytesseract import Output
import matplotlib.pyplot as plt
import camelot.io as camelot
from utils.boundingbox import BoundingBoxes
from utils.text import Text
import re
from spacy import displacy

/Users/eminasljivic/opt/miniconda3/envs/leoenv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-12-04 21:00:25.757157: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_dir = r'input/train'
test_dir = r'input/test'
fields_dir = r'input/fields'
field_names = {'company-name', 'company-address', 'invoice-id', 'invoice-date', 'total-value', 'item-header', 'qty-header', 'rate-header', 'total-header'}

testFilePath = train_dir+'/'+os.listdir(train_dir)[3]

In [3]:
bounding_boxes = BoundingBoxes()

# Train Named Entity Recognizer

In [5]:
fields = os.listdir(fields_dir)
invoices = os.listdir(train_dir)

In [6]:
print(len(fields))
print(len(invoices))

15
15


In [7]:
set_fields  = set(map(lambda f: f.split('.')[0], fields))
set_invoices = set(map(lambda f: f.split('.')[0], invoices))

training_set = list(set_fields.intersection(set_invoices))

In [9]:
data = pd.DataFrame(columns=["filename", "text"])

data["filename"] = training_set

data_text = []
for file in data["filename"]:
    data_text.append(get_text(f'{train_dir}/{file}.pdf'))
data["text"] = data_text

ent_list = []
for file in data["filename"]:
    with open(f'{fields_dir}/{file}.json') as f:
        entity_dict = json.load(f)
        ent_list.append(entity_dict)
data["entity_dictionary"] = ent_list

In [10]:
similar_chars = [
    ['b', '6', 'G'],
    ['c', 'C', '\('],
    ['q','g'],
    ['0', 'O'],
    ['S', '5', '9'],
    ['V', 'U', 'v', 'u'],
    ['Z','2'],
    ['B','ß'],
    ['e', 'c'],
    [' ', '..']
]
def similar_looking_chars(str):
    for similar_char in similar_chars:
        if str in similar_char:
            return similar_char
            
def search_by_one_char_off(str, text, entity):
    for i in range(0,len(str)):
        catch_total = re.search((str[:i] + '.' + str[i + 1:]), text)
        if catch_total != None:
            ent_tup = (catch_total.span()[0], catch_total.span()[1], entity)
            return ent_tup

def search_with_similar_chars(str, text, entity):
    temp_value = ''
    for i in range(0,len(str)):
        similar_chars = similar_looking_chars(str[i])
        if similar_chars:
            temp_value += '('+'|'.join(similar_chars)+')'
        else:
            temp_value += str[i]
    catch_total = re.search(temp_value, text)
    if catch_total != None:
        ent_tup = (catch_total.span()[0], catch_total.span()[1], entity)
        return ent_tup

In [32]:
training_data = []
id_ent = []

nlp_match = spacy.load('en_core_web_sm')
matcher = PhraseMatcher(nlp_match.vocab)
for index, row in data.iterrows():
    ent_dic = row["entity_dictionary"]
    ent = []
    phrases = list(ent_dic.values())
    patterns = [nlp_match.make_doc(phrase) for phrase in phrases]
    matcher.add("EntityList", None, *patterns)

    
    ent_set = field_names
    for entity in field_names:
        value = ent_dic[entity]
        text = row["text"]
        catch_total = re.search(value, text)
        if catch_total != None:
            ent_tup = (catch_total.span()[0], catch_total.span()[1], entity)
            ent.append(ent_tup)
        else:
            temp = search_by_one_char_off(value, text, entity)
            if not temp:
                temp = search_with_similar_chars(value, text, entity)
            if temp:
                ent.append(temp)

    id_ent.append(len(ent))
    training_data.append({"entities": ent, "text":row["text"]})

In [33]:
nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin() # create a DocBin object

for training_example in training_data: 
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text) 
    ents = []
    for start, end, label in labels:
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity", label)
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents 
    doc_bin.add(doc)

doc_bin.to_disk("model/training_data.spacy") # save the docbin object

# Testing NER

In [40]:
nlp_ner = spacy.load("model/model-best")

ValueError: [E961] Found non-serializable Python object in config. Configs should only include values that can be serialized to JSON. If you need to pass models or other objects to your component, use a reference to a registered function or initialize the object in your component.

{'data':    filename                                               text  \
0    INV006     \n Invoice \n FROM TO \n HTL Leonding Rechn...   
1    INV015     \n Invoice \n FROM TO \n HTL Leonding Rechn...   
2    INV001     \n Invoice \n FROM TO \n HTL Leonding Unbek...   
3    INV003     \n Invoice \n FROM TO \n HTL Leonding Unbek...   
4    INV004        \n    Subtotal Ust(20%) Total Invoice EU...   
5    INV011     \n Invoice \n FROM TO \n HTL Leonding Teste...   
6    INV012     \n Invoice \n FROM TO \n HTL Leonding Rechn...   
7    INV009     \n Invoice \n FROM TO \n HTL Leonding FoodT...   
8    INV002     \n Invoice \n FROM TO \n HTL Leonding FoodT...   
9    INV008     \n Invoice \n FROM TO \n HTL Leo

In [35]:
test_files = os.listdir(test_dir)
test_files

['INV016.pdf', 'INV017.pdf', 'INV020.pdf', 'INV019.pdf', 'INV018.pdf']

In [36]:
len(test_files)

5

In [37]:
test_data = pd.DataFrame(columns=["filename", "text"])

test_data["filename"] = test_files

data_text = []
for file in test_data["filename"]:
    data_text.append(get_text(f'{test_dir}/{file}'))
test_data["text"] = data_text

In [38]:
test_data['text'][0]

'   \n Invoice \n FROM TO \n HTL Leonding Rechnungsbearbeiter e.U. \n LimesstraBe 12/14 Rechnung Bearbeiter \n 4060 Leonding RechnungsstraBe 19 \n Austria Linz 4020 \n invoice@htl-leonding.ac.at Austria \n rechnung@berabeiter.at \n Invoice No.: INVO16 \n Due: Feb 28th, 2023 \n Invoice Date: Nov 28th, 2022 \n Item HRS/QTY Rate Tax Subtotal \n Docker Anbindung 1 600.00 Ust(20%) EUR 600.00 \n Kubernetes Anbdinung 1 700.90 Ust(20%) EUR 700.90 \n Invoice Summary \n Subtotal EUR 1,300.90 \n Ust(20%) EUR 260.18 \n Total EUR 1,561.08 \n'

In [39]:
for index, row in test_data.iterrows():
    op_dict = {}
    doc = nlp_ner(row["text"])
    for ent in doc.ents:      #identifying the entities using the trained model
        op_dict[ent.label_] = ent.text
  
    print("Entities", op_dict)

Entities {'company-name': 'Rechnungsbearbeiter e.U.', 'company-address': 'RechnungsstraBe 19', 'invoice-id': 'INVO16', 'invoice-date': 'Nov 28th, 2022', 'item-header': 'Item', 'qty-header': 'HRS/QTY', 'rate-header': 'Rate', 'total-header': 'Subtotal', 'total-value': 'EUR 1,561.08'}
Entities {'company-name': 'FoodTable', 'company-address': 'BundesstraBe 8/5', 'invoice-id': 'INV017', 'invoice-date': 'Nov 28th, 2022', 'item-header': 'Item', 'qty-header': 'HRS/QTY', 'rate-header': 'Rate', 'total-header': 'Subtotal', 'total-value': 'EUR 6,606.00'}
Entities {'company-name': 'Unbekanntes Unternehmen KG', 'company-address': 'MusterstraBe 1', 'invoice-id': 'INV020', 'invoice-date': 'Nov 28th, 2022', 'item-header': 'Item', 'qty-header': 'HRS/QTY', 'rate-header': 'Rate', 'total-header': 'Subtotal', 'total-value': 'EUR 18,009.00'}
Entities {'company-name': 'Unbekanntes Unternehmen KG', 'company-address': 'MusterstraBe 1', 'invoice-id': 'INVO19', 'invoice-date': 'Nov 28th, 2022', 'item-header': 'It